# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# aggregating amount_spent by customer
customer_expenses = orders.groupby(["CustomerID"]).agg({'amount_spent':sum}).reset_index()

In [5]:
# filtering for customer with aggregated expense above the 95th percentile and above the 75th perceltile
pct_95 = customer_expenses['amount_spent'].quantile(0.95)
pct_75 = customer_expenses['amount_spent'].quantile(0.75)

In [6]:
# creating a function for labelign customers based on spend
def customer_type(row):
    if((row["amount_spent"] > pct_75) and(row["amount_spent"] < pct_95) ):
       return "Preferred Customer"
    elif ((row ["amount_spent"] > pct_95)):
       return "VIP Customer"
    else:
       return "Other"

customer_expenses.apply(customer_type, axis=1).head()

0          VIP Customer
1    Preferred Customer
2    Preferred Customer
3    Preferred Customer
4                 Other
dtype: object

In [8]:
# adding a new column "Label" to a df customer_expenses
customer_expenses['Label'] = customer_expenses.apply(customer_type, axis=1)
customer_expenses.head()

,CustomerID,amount_spent,Label
0,12346,77183.60,VIP Customer
1,12347,4310.00,Preferred Customer
2,12348,1797.24,Preferred Customer
3,12349,1757.55,Preferred Customer
4,12350,334.40,Other


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [10]:
# merginf data from customer_expenses with orders to use the column "Label" on the original dataset
merge_label = pd.merge(left=orders, right=customer_expenses, how="inner", left_on="CustomerID", right_on ="CustomerID")
merge_label.head()

#using grouping to see number of VIP customers by country
merge_label[merge_label["Label"] == "VIP Customer"].groupby(["Country"]).agg({'CustomerID':pd.Series.nunique}) \
.sort_values(by=["CustomerID"],ascending=False).reset_index()

,Country,CustomerID
0,United Kingdom,177
1,Germany,10
2,France,9
3,Switzerland,3
4,Spain,2
5,Portugal,2
6,Australia,2
7,EIRE,2
8,Japan,2
9,Finland,1


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [12]:
#using grouping to see number of VIP and Preferred customers by country
merge_label[merge_label["Label"].isin(["VIP Customer", "Preferred Customer"])].groupby(["Country"]) \
.agg({'CustomerID':pd.Series.nunique}) \
.sort_values(by=["CustomerID"],ascending=False).reset_index()

,Country,CustomerID
0,United Kingdom,932
1,Germany,39
2,France,29
3,Belgium,12
4,Switzerland,9
5,Spain,9
6,Portugal,7
7,Norway,7
8,Italy,5
9,Finland,5
